In [168]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import metrics
import pandas_datareader.data as web


In [169]:
#Read stock data use pandas_datareader.data from web
# Get Stock Data
stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']
stk_data = web.DataReader(stk_tickers, 'yahoo')
ccy_data = web.DataReader(ccy_tickers, 'fred')
idx_data = web.DataReader(idx_tickers, 'fred')

In [170]:
# Select columns
base = stk_data.loc[:, ('Adj Close', 'MSFT')]
X1 = stk_data.loc[:, ('Adj Close', ('GOOGL', 'IBM'))]
X2 = ccy_data
X3 = idx_data
print(X1)
print(X2)
print(X3)

Attributes   Adj Close            
Symbols          GOOGL         IBM
Date                              
2017-09-15   46.764500  109.150810
2017-09-18   46.487499  108.947289
2017-09-19   46.842999  108.826714
2017-09-20   47.376999  109.949699
2017-09-21   47.377499  109.482414
...                ...         ...
2022-09-09  110.650002  129.190002
2022-09-12  110.860001  130.660004
2022-09-13  104.320000  127.250000
2022-09-14  105.000000  127.690002
2022-09-15  103.077499  125.320000

[1259 rows x 2 columns]
            DEXJPUS  DEXUSUK
DATE                        
2017-09-18   111.48   1.3485
2017-09-19   111.49   1.3517
2017-09-20   111.50   1.3564
2017-09-21   112.30   1.3576
2017-09-22   112.01   1.3531
...             ...      ...
2022-09-05      NaN      NaN
2022-09-06   142.95   1.1549
2022-09-07   144.39   1.1473
2022-09-08   144.05   1.1489
2022-09-09   142.44   1.1600

[1300 rows x 2 columns]
              SP500      DJIA  VIXCLS
DATE                                 
2017-09

In [171]:
#Standardized data (X1, X2, X3) with kept index (date)
standard_scaler = preprocessing.StandardScaler()
sX1 = standard_scaler.fit_transform(X1.values)
sX2 = standard_scaler.fit_transform(X2.values)
sX3 = standard_scaler.fit_transform(X3.values)
print(type(sX1))
sX1

<class 'numpy.ndarray'>


array([[-1.14793319, -0.577624  ],
       [-1.15668843, -0.59643565],
       [-1.14545206, -0.60758048],
       ...,
       [ 0.67124038,  1.09529664],
       [ 0.69273334,  1.13596637],
       [ 0.6319683 ,  0.91690537]])

In [172]:
sX1 = pd.DataFrame(index = X1.index,data = sX1,columns=X1.columns)
sX2 = pd.DataFrame(index = X2.index,data = sX2,columns=X2.columns)
sX3 = pd.DataFrame(index = X3.index,data = sX3,columns=X3.columns)
sX1

Attributes Adj Close          
Symbols        GOOGL       IBM
Date                          
2017-09-15 -1.147933 -0.577624
2017-09-18 -1.156688 -0.596436
2017-09-19 -1.145452 -0.607580
2017-09-20 -1.128574 -0.503782
2017-09-21 -1.128558 -0.546974
...              ...       ...
2022-09-09  0.871315  1.274612
2022-09-12  0.877952  1.410486
2022-09-13  0.671240  1.095297
2022-09-14  0.692733  1.135966
2022-09-15  0.631968  0.916905

[1259 rows x 2 columns]

In [173]:
X1['Adj Close']

Symbols,GOOGL,IBM
Date,,
2017-09-15,46.764500,109.150810
2017-09-18,46.487499,108.947289
2017-09-19,46.842999,108.826714
2017-09-20,47.376999,109.949699
2017-09-21,47.377499,109.482414
...,...,...
2022-09-09,110.650002,129.190002
2022-09-12,110.860001,130.660004
2022-09-13,104.320000,127.250000


In [174]:
# Calculate ความแตกต่างของค่า ราคา 'Adj Close', 'MSFT’)ย้อนหลัง return_period วัน
return_period = 1 #predict 1 วันข้างหน้า
Y = base.shift(-return_period)
X4_3DT = base.diff(3*return_period).shift(-3*return_period)
X4_6DT = base.diff(6*return_period).shift(-6*return_period)
X4_12DT = base.diff(12*return_period).shift(-12*return_period)
X4 = pd.concat([X4_3DT, X4_6DT, X4_12DT], axis=1)
X4.columns = ['MSFT_3DT', 'MSFT_6DT', 'MSFT_12DT']
X4 = pd.DataFrame(standard_scaler.fit_transform(X4.values), index = X4.index,columns=X4.columns)

# Forming Dataset
X = pd.concat([sX1, sX2, sX3, X4], axis=1)
dataset = pd.concat([Y, X], axis=1)
print(type(dataset))


<class 'pandas.core.frame.DataFrame'>


In [175]:
dataset.dropna(inplace=True)
dataset

,"(Adj Close, MSFT)","(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-18,70.887177,-1.156688,-0.596436,-0.034125,0.632440,-1.265640,-1.502526,-1.182579,-0.231866,-0.342048,-0.209676
2017-09-19,70.417366,-1.145452,-0.607580,-0.032778,0.687607,-1.261609,-1.492996,-1.179130,-0.245060,-0.304829,-0.122054
2017-09-20,69.731415,-1.128574,-0.503782,-0.031431,0.768632,-1.259303,-1.482900,-1.225117,-0.352268,-0.242404,-0.075616
2017-09-21,69.919342,-1.128558,-0.546974,0.076321,0.789320,-1.270381,-1.495791,-1.237764,-0.231866,-0.080335,0.013763
2017-09-22,68.838745,-1.135338,-0.556031,0.037261,0.711742,-1.268032,-1.498120,-1.246961,-0.167540,-0.089940,-0.003761
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-23,275.790009,0.972774,1.787603,3.316975,-2.196569,1.090340,1.053012,0.422379,-1.540819,-2.026554,-1.285640
2022-08-24,278.850006,0.967401,1.648032,3.392402,-2.268975,1.107798,1.067420,0.274069,-1.928729,-2.080216,-1.020807
2022-08-25,268.089996,1.060958,1.717355,3.380280,-2.267251,1.192403,1.145343,0.154502,-2.862526,-3.025658,-2.673211
2022-08-26,265.230011,0.861516,1.384605,3.412605,-2.310350,0.987292,0.901734,0.589083,-1.237158,-2.009945,-1.648384


In [176]:
dataset.describe

<bound method NDFrame.describe of             (Adj Close, MSFT)  (Adj Close, GOOGL)  (Adj Close, IBM)   DEXJPUS  \
2017-09-18          70.887177           -1.156688         -0.596436 -0.034125   
2017-09-19          70.417366           -1.145452         -0.607580 -0.032778   
2017-09-20          69.731415           -1.128574         -0.503782 -0.031431   
2017-09-21          69.919342           -1.128558         -0.546974  0.076321   
2017-09-22          68.838745           -1.135338         -0.556031  0.037261   
...                       ...                 ...               ...       ...   
2022-08-23         275.790009            0.972774          1.787603  3.316975   
2022-08-24         278.850006            0.967401          1.648032  3.392402   
2022-08-25         268.089996            1.060958          1.717355  3.380280   
2022-08-26         265.230011            0.861516          1.384605  3.412605   
2022-08-29         262.970001            0.832438          1.378134  3.6375

In [177]:
Y = pd.DataFrame(dataset[dataset.columns[0]].reset_index(drop=True))
X = pd.DataFrame(dataset[dataset.columns[1:]].reset_index(drop=True))
print(Y)
print(X)

       Adj Close
            MSFT
0      70.887177
1      70.417366
2      69.731415
3      69.919342
4      68.838745
...          ...
1224  275.790009
1225  278.850006
1226  268.089996
1227  265.230011
1228  262.970001

[1229 rows x 1 columns]
      (Adj Close, GOOGL)  (Adj Close, IBM)   DEXJPUS   DEXUSUK     SP500  \
0              -1.156688         -0.596436 -0.034125  0.632440 -1.265640   
1              -1.145452         -0.607580 -0.032778  0.687607 -1.261609   
2              -1.128574         -0.503782 -0.031431  0.768632 -1.259303   
3              -1.128558         -0.546974  0.076321  0.789320 -1.270381   
4              -1.135338         -0.556031  0.037261  0.711742 -1.268032   
...                  ...               ...       ...       ...       ...   
1224            0.972774          1.787603  3.316975 -2.196569  1.090340   
1225            0.967401          1.648032  3.392402 -2.268975  1.107798   
1226            1.060958          1.717355  3.380280 -2.267251  1.1924

In [178]:
X.corr()

,"(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
"(Adj Close, GOOGL)",1.000000,0.601793,0.392870,0.266160,0.985415,0.960172,0.153017,-0.053557,-0.076005,-0.102293
"(Adj Close, IBM)",0.601793,1.000000,0.570644,0.228632,0.608929,0.661024,-0.285608,-0.084682,-0.132809,-0.173998
DEXJPUS,0.392870,0.570644,1.000000,-0.365275,0.350181,0.343815,0.050443,-0.075823,-0.114391,-0.161840
DEXUSUK,0.266160,0.228632,-0.365275,1.000000,0.278266,0.322238,-0.327849,0.004784,0.013394,0.026486
SP500,0.985415,0.608929,0.350181,0.278266,1.000000,0.986195,0.108280,-0.060321,-0.088016,-0.120863
DJIA,0.960172,0.661024,0.343815,0.322238,0.986195,1.000000,-0.005292,-0.062945,-0.091934,-0.123961
VIXCLS,0.153017,-0.285608,0.050443,-0.327849,0.108280,-0.005292,1.000000,0.059809,0.081869,0.110009
MSFT_3DT,-0.053557,-0.084682,-0.075823,0.004784,-0.060321,-0.062945,0.059809,1.000000,0.688943,0.491060
MSFT_6DT,-0.076005,-0.132809,-0.114391,0.013394,-0.088016,-0.091934,0.081869,0.688943,1.000000,0.680511
MSFT_12DT,-0.102293,-0.173998,-0.161840,0.026486,-0.120863,-0.123961,0.110009,0.491060,0.680511,1.000000


In [187]:
lower = pd.DataFrame(np.tril(X.corr(),-1),columns = X.columns)
to_drop = [column for column in lower.columns if any(lower[column] > 0.9)]
to_drop

[('Adj Close', 'GOOGL'), 'SP500']

In [188]:
X.drop(columns=to_drop)

,"(Adj Close, IBM)",DEXJPUS,DEXUSUK,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
0,-0.596436,-0.034125,0.632440,-1.502526,-1.182579,-0.231866,-0.342048,-0.209676
1,-0.607580,-0.032778,0.687607,-1.492996,-1.179130,-0.245060,-0.304829,-0.122054
2,-0.503782,-0.031431,0.768632,-1.482900,-1.225117,-0.352268,-0.242404,-0.075616
3,-0.546974,0.076321,0.789320,-1.495791,-1.237764,-0.231866,-0.080335,0.013763
4,-0.556031,0.037261,0.711742,-1.498120,-1.246961,-0.167540,-0.089940,-0.003761
...,...,...,...,...,...,...,...,...
1224,1.787603,3.316975,-2.196569,1.053012,0.422379,-1.540819,-2.026554,-1.285640
1225,1.648032,3.392402,-2.268975,1.067420,0.274069,-1.928729,-2.080216,-1.020807
1226,1.717355,3.380280,-2.267251,1.145343,0.154502,-2.862526,-3.025658,-2.673211
1227,1.384605,3.412605,-2.310350,0.901734,0.589083,-1.237158,-2.009945,-1.648384
